In [ ]:
include("choice_constructor.jl")
include("choice_types.jl")
include("choice_memhandler.jl")
include("choice_core.jl")

In [ ]:
function load(filename, clean="y")
    file = open(filename, "r")
    vcal = readlines(file)
    close(file)
    if clean == "y"
        processed = []
        for line in vcal
            push!(processed, replace(line, r"\t" => ""))
        end
        return processed
    end
    return vcal
end

function separate(vcal)
    blocks = []
    block = []
    for line in vcal
        if line == ""
            push!(blocks, block)
            block = []
            continue
        end
        push!(block, line)
    end
    push!(blocks, block)
    return blocks
end

function getMetadata(block)
    memargs = (Int)[]
    keywords = (String)[]
    for line in block
        if occursin("Const", line)
            push!(keywords, replace(line, "Const " => ""))
        else
            line = replace(line, r"\s+" => "")
            for part in split(line, ",")
                push!(memargs, parse(Int, part))
            end
        end
    end
    return memargs, keywords
end

function getFunctionKeys(blocks)
    fkeywords = (String)[]
    for block in blocks
        push!(fkeywords, block[1][1:findfirst(" ", block[1])[1]-1])
    end
    return fkeywords
end

function parseLine(line, keywords=[], fkeywords=[], scope=nothing)
    line = replace(line, r"\s+" => " ")
    outputs = nothing
    inputs = nothing
    conditionals = nothing
    operation = nothing
    condition = nothing
    if occursin(" = ", line)
        s1, s2 = split(line, " = ")
        s1 = replace(s1, r"\s+" => "")
        s1 = replace(s1, "(" => "")
        s1 = replace(s1, ")" => "")
        outputs = [String(var) for var in split(s1, ",")]
        if occursin(" if ", s2)
            s2, s1 = split(s2, " if ")
            s = split(s1, " ")
            conditionals = [String(s[1])]
            condition = s[2]
        end
        if occursin("(", s2)
            s1, s2 = split(s2, "(")
            operation = String(s1) in fkeywords ? Flabel(String(s1)) : String(s1)
            s2 = replace(s2, r"\s+" => "")
            s2 = replace(s2, ")" => "")
            inputs = [String(var) for var in split(s2, ",")]
        else
            inputs = [String(replace(s2, r"\s+" => ""))]
            operation = ""
        end
    elseif replace(line, r"\s+" => "") == "break"
        return Line(4, nothing, nothing, nothing, nothing)
    elseif occursin("repeat ", line)
        s2 = line
        if occursin(" if ", line)
            s2, s1 = split(line, " if ")
            s = split(s1, " ")
            conditionals = [String(s[1])]
            condition = s[2]
        end
        operation = Flabel(scope)
        s2 = replace(s2, "repeat" => "")
        if occursin("(", s2)
            s1, s2 = split(s2, "(")
            s2 = replace(s2, r"\s+" => "")
            s2 = replace(s2, ")" => "")
            inputs = [String(var) for var in split(s2, ",")]
        else
            inputs = [String(replace(s2, r"\s+" => ""))]
        end
    else
        s2 = line
        if occursin(" if ", line)
            s2, s1 = split(line, " if ")
            s = split(s1, " ")
            conditionals = [String(s[1])]
            condition = s[2]
        end
        if occursin("(", s2)
            s1, s2 = split(s2, "(")
            operation = String(s1) in fkeywords ? Flabel(String(s1)) : String(s1)
            s2 = replace(s2, r"\s+" => "")
            s2 = replace(s2, ")" => "")
            inputs = [String(var) for var in split(s2, ",")]
        else
            inputs = [String(replace(s2, r"\s+" => ""))]
            operation = ""
        end
    end
    inputs = inputs == nothing ? nothing : [var in keywords ? Const(var) : var for var in inputs]
    inputs = inputs == [""] ? nothing : inputs
    return Line(operation, condition, outputs, inputs, conditionals)
end

function parseVcal(vcal)
    vcal = separate(vcal)
    memargs, keywords = getMetadata(vcal[1])
    fkeywords = getFunctionKeys(vcal[2:end])
end

In [ ]:
vcal = load("test.vcal")
vcal = separate(vcal)
getMetadata(vcal[1])
getFunctionKeys(vcal[2:end])

l = "repeat (a, b) if x > 0"
parseLine(l, ["read"], ["f"], "f")
#s1, s2 = split(l, "=")

In [ ]:
lh = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line("", "=", [1], [1], [3]),
    Line("dec", nothing, [3], [3], nothing),
    Line("", "=", [1], [2], [3]),
    Line(1, nothing, nothing, [1, 2, 3], nothing)
]
h = Function("h", nothing, nothing, lh)
display(h.lines)

lg = [
    Line(0, nothing, [1, 2, 3], nothing, nothing),
    Line("+", nothing, [1], [1, 2], nothing),
    Line("", nothing, [4], [2], nothing),
    Line("", nothing, [2], [1], nothing),
    Line("", nothing, [1], [4], nothing),
    Line("dec", nothing, [3], [3], nothing),
    Line(Flabel("g"), ">=", nothing, [1, 2, 3], [3]),
    Line(1, nothing, nothing, [1, 2, 3], nothing)
]
g = Function("g", nothing, nothing, lg)
display(g.lines)

lmain = [
    Line(0, nothing, [], nothing, nothing),
    Line("", nothing, [1], [Const("read")], nothing),
    Line("", nothing, [2], [Const("read")], nothing),
    Line("", nothing, [3], [Const("read")], nothing),
    Line(Flabel("h"), nothing, [1, 2, 3], [1, 2, 3], nothing),
    Line(Flabel("g"), ">", [1, 2, 3], [1, 2, 3], [3]),
    Line("write", nothing, nothing, [1], nothing),
    Line("write", nothing, nothing, [2], nothing),
    Line("write", nothing, nothing, [3], nothing),
    Line(1, nothing, nothing, [], nothing)
]
main = Function("main", [], [], lmain)
display(main.lines)

functions = Dict()
push!(functions, "g" => g)
push!(functions, "h" => h)

In [ ]:
preasm = choose(main, functions)
create(preasm, "w")

In [ ]:
preasm = [Line("break", nothing, nothing, nothing, nothing)]
create(preasm, "w")